## RI Model LN Auto V2

In [2]:
!pip install xgboost

  Using cached xgboost-2.0.3-py3-none-manylinux2014_x86_64.whl.metadata (2.0 kB)
Using cached xgboost-2.0.3-py3-none-manylinux2014_x86_64.whl (297.1 MB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import json
import logging
import xgboost as xgb
import ast
import itertools
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2
import os
import sys

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [4]:
sys.path.append('/root/Unified_LN_V2/')
from Utilities import *

In [5]:
Train = pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_Train_RI_Data_220324.csv')
Test = pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_Test_RI_Data_220324.csv')

Train.shape, Test.shape

((490406, 345), (210174, 345))

In [6]:
req_cols= ['assetpropevercount', 'sourcenonderogcount06month', 'addronfilecount', 'addrinputcountyratio', 'assetpropnewestsaleprice', 'addrcurrenttaxvalue', 'addrinputavmvalue', 'addrinputavmvalue60month', 'addrinputlastsaleprice', 'addrinputtaxmarketvalue', 'sourcecredheadertimeoldest', 'addrinputphonecount', 'assetpropcurrenttaxtotal', 'addrcurrentavmvalue', 'assetproppurchasetimeoldest', 'addrpreviouslengthofres', 'addrinputlengthofres', 'addrcurrenttaxmarketvalue', 'addrprevioustimenewest', 'addrcurrenttractratio', 'sourcenonderogcount12month', 'assetpropcurrentcount', 'addrinputsubjectcount', 'sourcenonderogcount', 'addrinputtaxvalue', 'assetpropeversoldcount', 'businessassociationtimeoldest', 'addrinputtractratio', 'derogtimenewest', 'addrchangecount24month', 'addrinputtimelastsale', 'educationinstitutionprivate', 'addrinputblockratio', 'businesstitleleadership', 'addrcurrentlengthofres', 'addrcurrentavmratio12monthprior', 'addrcurrentblockratio', 'derogcount', 'profliccount', 'addrinputavmratio60monthprior', 'addrcurrentcountyratio', 'assetpropsaletimeoldest', 'subjectrecordtimeoldest', 'addrcurrenttimenewest', 'proflictypecategory', 'criminalnonfelonytimenewest', 'lienjudgmentdollartotal', 'addrcurrentlastsalesprice', 'addrinputtimenewest', 'evictiontimenewest', 'bankruptcycount', 'addrcurrenttimelastsale', 'addrinputtimeoldest', 'addrlastmovetaxratiodiff', 'criminalnonfelonycount', 'sourcenonderogcount03month', 'addrchangecount03month', 'addrchangecount06month', 'addrchangecount12month', 'addrchangecount60month', 'addrcurrentavmratio60monthprior', 'addrcurrentavmvalue12month', 'addrcurrentavmvalue60month', 'addrcurrentcorrectional', 'addrcurrentdeedmailing', 'addrcurrentphoneservice', 'addrcurrentsubjectowned', 'addrcurrenttimeoldest', 'addrinputavmratio12monthprior', 'addrinputavmvalue12month', 'addrinputdeedmailing', 'addrinputphoneservice', 'addrinputproblems', 'addrinputsubjectowned', 'addrlastmoveecontrajectory', 'addronfilecollege', 'addronfilecorrectional', 'addronfilehighrisk', 'addrpreviouscorrectional', 'addrprevioussubjectowned', 'addrprevioustimeoldest', 'alertregulatorycondition', 'assetownership', 'assetpersonal', 'assetpersonalcount', 'assetprop', 'assetpropnewestmortgagetype', 'assetproppurchasecount12month', 'assetproppurchasetimenewest', 'assetpropsalepurchaseratio', 'assetpropsaletimenewest', 'assetpropsoldcount12month', 'bankruptcychapter', 'bankruptcycount24month', 'bankruptcydismissed24month', 'bankruptcystatus', 'bankruptcytimenewest', 'businessassociation', 'criminalfelonycount', 'criminalfelonycount12month', 'criminalfelonytimenewest', 'criminalnonfelonycount12month', 'derogcount12month', 'educationattendance', 'educationevidence', 'educationinstitutionrating', 'educationprogramattended', 'evictioncount', 'evictioncount12month', 'inquiryauto12month', 'inquirybanking12month', 'inquirycollections12month', 'inquirynonshortterm12month', 'inquiryshortterm12month', 'inquirytelcom12month', 'lienjudgmentcount', 'lienjudgmentcount12month', 'lienjudgmentcourtcount', 'lienjudgmentforeclosurecount', 'lienjudgmentothercount', 'lienjudgmentsmallclaimscount', 'lienjudgmenttaxcount', 'lienjudgmenttimenewest', 'shorttermloanrequest', 'shorttermloanrequest12month', 'shorttermloanrequest24month', 'sourcecredheadertimenewest', 'sourcevoterregistration', 'subjectnewestrecord12month', 'subjectrecordtimenewest']
len(req_cols)

130

In [7]:
all_cols= ['unique_id', 'client', 'app_date', 'approve_flag', 'funded_flag', 'fico_pb', 'fico_cb', 'bad_flag', 'product', 'auto_score', 'bankcard_score', 'Race', 'Gender', 'age_cuts'] + req_cols
len(all_cols)

144

In [8]:
Train[all_cols].head()

,unique_id,client,app_date,approve_flag,funded_flag,fico_pb,fico_cb,bad_flag,product,auto_score,bankcard_score,Race,Gender,age_cuts,assetpropevercount,sourcenonderogcount06month,addronfilecount,addrinputcountyratio,assetpropnewestsaleprice,addrcurrenttaxvalue,addrinputavmvalue,addrinputavmvalue60month,addrinputlastsaleprice,addrinputtaxmarketvalue,sourcecredheadertimeoldest,addrinputphonecount,assetpropcurrenttaxtotal,addrcurrentavmvalue,assetproppurchasetimeoldest,addrpreviouslengthofres,addrinputlengthofres,addrcurrenttaxmarketvalue,addrprevioustimenewest,addrcurrenttractratio,sourcenonderogcount12month,assetpropcurrentcount,addrinputsubjectcount,sourcenonderogcount,addrinputtaxvalue,assetpropeversoldcount,businessassociationtimeoldest,addrinputtractratio,derogtimenewest,addrchangecount24month,addrinputtimelastsale,educationinstitutionprivate,addrinputblockratio,businesstitleleadership,addrcurrentlengthofres,addrcurrentavmratio12monthprior,addrcurrentblockratio,derogcount,profliccount,addrinputavmratio60monthprior,addrcurrentcountyratio,assetpropsaletimeoldest,subjectrecordtimeoldest,addrcurrenttimenewest,proflictypecategory,criminalnonfelonytimenewest,lienjudgmentdollartotal,addrcurrentlastsalesprice,addrinputtimenewest,evictiontimenewest,bankruptcycount,addrcurrenttimelastsale,addrinputtimeoldest,addrlastmovetaxratiodiff,criminalnonfelonycount,sourcenonderogcount03month,addrchangecount03month,addrchangecount06month,addrchangecount12month,addrchangecount60month,addrcurrentavmratio60monthprior,addrcurrentavmvalue12month,addrcurrentavmvalue60month,addrcurrentcorrectional,addrcurrentdeedmailing,addrcurrentphoneservice,addrcurrentsubjectowned,addrcurrenttimeoldest,addrinputavmratio12monthprior,addrinputavmvalue12month,addrinputdeedmailing,addrinputphoneservice,addrinputproblems,addrinputsubjectowned,addrlastmoveecontrajectory,addronfilecollege,addronfilecorrectional,addronfilehighrisk,addrpreviouscorrectional,addrprevioussubjectowned,addrprevioustimeoldest,alertregulatorycondition,assetownership,assetpersonal,assetpersonalcount,assetprop,assetpropnewestmortgagetype,assetproppurchasecount12month,assetproppurchasetimenewest,assetpropsalepurchaseratio,assetpropsaletimenewest,assetpropsoldcount12month,bankruptcychapter,bankruptcycount24month,bankruptcydismissed24month,bankruptcystatus,bankruptcytimenewest,businessassociation,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount12month,derogcount12month,educationattendance,educationevidence,educationinstitutionrating,educationprogramattended,evictioncount,evictioncount12month,inquiryauto12month,inquirybanking12month,inquirycollections12month,inquirynonshortterm12month,inquiryshortterm12month,inquirytelcom12month,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentcourtcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmentsmallclaimscount,lienjudgmenttaxcount,lienjudgmenttimenewest,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,sourcecredheadertimenewest,sourcevoterregistration,subjectnewestrecord12month,subjectrecordtimenewest
0,61929,Arkansas,2020-05-21,1.0,0.0,726.0,735.0,NaN,Auto,707.0,738.0,white,M,Age below 62,40.0,3.0,35.0,0.31,-1.0,0.0,43597.0,-1.0,-1.0,0.0,237.0,0.0,551404.0,43597.0,89.0,1.0,142.0,0.0,89.0,0.23,3.0,40.0,0.0,5.0,0.0,0.0,-1.0,0.23,35.0,0.0,-1.0,-1.0,0.29,-1.0,142.0,-1.00,0.29,1.0,0.0,-1.00,0.31,-1.0,303.0,1.0,-1.0,-1.0,42466.0,-1.0,1.0,-1.0,0.0,-1.0,143.0,-1.0,0.0,3.0,0.0,0.0,0.0,0.0,-1.00,-1.0,-1.0,0.0,1.0,0.0,0.0,143.0,-1.00,-1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,89.0,1.0,1.0,1.0,3.0,1.0,-1.0,0.0,89.0,0.00,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,35.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1,TCI196142,Numerica,2017-12-19,1.0,1.0,802.0,NaN,0.0,Auto,792.0,826.0,white,M,Age below 62,3.0,2.0,7.0,1.51,167200.0,0.0,301616.0,257392.0,-1.0,0.0,324.0,0.0,238800.0,301616.0,244.0,114.0,106.0,0.0,4

In [9]:
Train['Race'].value_counts(dropna= False), Train['Gender'].value_counts(dropna= False), Train['age_cuts'].value_counts(dropna= False)

(Race
 white       270333
 missing     132224
 black        42559
 hispanic     38530
 api           6108
 aian           555
 multiple        95
 others           2
 Name: count, dtype: int64,
 Gender
 M    268384
 F    185652
 I     36370
 Name: count, dtype: int64,
 age_cuts
 Age below 62       434592
 Age 62 and over     54323
 missing              1491
 Name: count, dtype: int64)

In [10]:
Test['Race'].value_counts(dropna= False), Test['Gender'].value_counts(dropna= False), Test['age_cuts'].value_counts(dropna= False)

(Race
 white       115715
 missing      56627
 black        18375
 hispanic     16499
 api           2663
 aian           261
 multiple        33
 others           1
 Name: count, dtype: int64,
 Gender
 M    114874
 F     79661
 I     15639
 Name: count, dtype: int64,
 age_cuts
 Age below 62       186033
 Age 62 and over     23506
 missing               635
 Name: count, dtype: int64)

### Base Model RI

In [11]:
!pip install hyperopt

  Using cached hyperopt-0.2.7-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
import numpy as np
import pandas as pd
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
import h2o
from h2o.estimators import H2OXGBoostEstimator
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
import warnings
warnings.filterwarnings('ignore')

In [13]:
### Starting H2o Session
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.2" 2019-01-15; OpenJDK Runtime Environment 18.9 (build 11.0.2+9); OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8lmuumop
  JVM stdout: /tmp/tmp8lmuumop/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp8lmuumop/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_unknownUser_gbo1be
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.486 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [14]:
Train= Train[all_cols]
Test= Test[all_cols]

Train.shape, Test.shape

((490406, 144), (210174, 144))

In [15]:
### Convert the Variable into categorical
var_to_cat= '''businesstitleleadership
proflictypecategory
addrcurrentdeedmailing
addrcurrentphoneservice
addrcurrentsubjectowned
addrinputdeedmailing
addrinputphoneservice
addrinputproblems
addrinputsubjectowned
addrlastmoveecontrajectory
addronfilecollege
addronfilecorrectional
addronfilehighrisk
addrprevioussubjectowned
alertregulatorycondition
assetownership
assetpersonal
assetprop
assetpropnewestmortgagetype
bankruptcychapter
bankruptcystatus
businessassociation
educationattendance
educationevidence
educationinstitutionrating
educationprogramattended
inquiryauto12month
inquirybanking12month
inquirycollections12month
inquirynonshortterm12month
inquiryshortterm12month
inquirytelcom12month
shorttermloanrequest
shorttermloanrequest12month
shorttermloanrequest24month
subjectnewestrecord12month'''.splitlines()
len(var_to_cat)

36

In [16]:
var_to_cat= list(set(var_to_cat))
len(var_to_cat)

36

In [17]:
Train[req_cols].isnull().sum()

assetpropevercount                 0
sourcenonderogcount06month         0
addronfilecount                    0
addrinputcountyratio               0
assetpropnewestsaleprice           0
addrcurrenttaxvalue                0
addrinputavmvalue                  0
addrinputavmvalue60month           0
addrinputlastsaleprice             0
addrinputtaxmarketvalue            0
sourcecredheadertimeoldest         0
addrinputphonecount                0
assetpropcurrenttaxtotal           0
addrcurrentavmvalue                0
assetproppurchasetimeoldest        0
addrpreviouslengthofres            0
addrinputlengthofres               0
addrcurrenttaxmarketvalue          0
addrprevioustimenewest             0
addrcurrenttractratio              0
sourcenonderogcount12month         0
assetpropcurrentcount              0
addrinputsubjectcount              0
sourcenonderogcount                0
addrinputtaxvalue                  0
assetpropeversoldcount             0
businessassociationtimeoldest      0
a

In [18]:
# for var in var_to_cat:
#     print(var)
#     print(Train[var].value_counts(dropna= False))

In [19]:
for var in var_to_cat:
    Train[var]= Train[var].astype(int).astype(str)
    Test[var]= Test[var].astype(int).astype(str)

In [20]:
Train['wt2']= 1
Test['wt2']= 1

train_h2o = h2o.H2OFrame(Train)
test_h2o = h2o.H2OFrame(Test)
print(Train.shape,train_h2o.shape,Test.shape,test_h2o.shape)
bad_col = 'approve_flag'
Train[bad_col].mean(),Test[bad_col].mean()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
(490406, 145) (490406, 145) (210174, 145) (210174, 145)


(0.589168974278455, 0.5897066240353231)

In [21]:
# S1,S2= train_test_split(Train, test_size= 0.4, random_state= 101)
# S1.shape, S2.shape

In [22]:
# S1['approve_flag'].mean(), S2['approve_flag'].mean()

In [23]:
# Train.groupby('client').agg(
#     ttd= ('unique_id', 'count'),
#     approved= ('approve_flag', 'sum'),
#     funded= ('funded_flag', 'sum'),
#     bad= ('bad_flag', 'sum'),
#     avg_fico= ('fico_pb', 'mean'),
#     avg_auto_score= ('auto_score', 'mean'),
#     min_app_date=  ('app_date', 'min'),
#     max_app_date= ('app_date', 'max')
# )

In [24]:
# S1.groupby('client').agg(
#     ttd= ('unique_id', 'count'),
#     approved= ('approve_flag', 'sum'),
#     funded= ('funded_flag', 'sum'),
#     bad= ('bad_flag', 'sum'),
#     avg_fico= ('fico_pb', 'mean'),
#     avg_auto_score= ('auto_score', 'mean'),
#     min_app_date=  ('app_date', 'min'),
#     max_app_date= ('app_date', 'max')
# )

In [25]:
# S2.groupby('client').agg(
#     ttd= ('unique_id', 'count'),
#     approved= ('approve_flag', 'sum'),
#     funded= ('funded_flag', 'sum'),
#     bad= ('bad_flag', 'sum'),
#     avg_fico= ('fico_pb', 'mean'),
#     avg_auto_score= ('auto_score', 'mean'),
#     min_app_date=  ('app_date', 'min'),
#     max_app_date= ('app_date', 'max')
# )

In [26]:
# sample= S1.copy()
# sample.shape

In [27]:
# del S1, S2

In [28]:
# sample_h2o = h2o.H2OFrame(sample)
# sample_h2o.shape

### Baseline

In [29]:
# config= pd.read_csv('/home/pradeep/000_LDA_March2024/configfile.csv')
unique_cols = ['unique_id', 'client']
wt_col = 'wt2'
folds = 10
# predictors = train_h2o.columns
# req_cols = config.dropna().Feature_Name.to_list()
model_cols = unique_cols + [wt_col] + [bad_col] + req_cols

# sample_h2o=train_h2o[model_cols+['age_cuts','Race','Gender']]#,'mmct_tag']]
# sample_h2o[bad_col] = sample_h2o[bad_col].asfactor()
# print(sample_h2o.shape)
# print(len(req_cols))

(490406, 137)
130


### Run the Code

In [39]:
xgbm_ri_auto_model = H2OXGBoostEstimator(
    ntrees=2000,    
    learn_rate=0.01,
    max_depth=2,    
    model_id="RI_Auto_LN_V2_1",
    distribution = "bernoulli",    
    seed=42,
    nfolds = folds)#,    ignored_columns = unique_cols)

In [40]:
xgbm_ri_auto_model.train(y=bad_col, x= req_cols, training_frame=train_h2o[model_cols], weights_column =wt_col)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: RI_Auto_LN_V2_1


Model Summary: 
    number_of_trees
--  -----------------
    2000

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.21364674753387844
RMSE: 0.4622193716557955
LogLoss: 0.6156456805843032
Mean Per-Class Error: 0.42758959524799944
AUC: 0.6989049229684253
AUCPR: 0.7542152027207817
Gini: 0.39780984593685065

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.39354477006738836
       0      1       Error    Rate
-----  -----  ------  -------  -------------------
0      41764  159710  0.7927   (159710.0/201474.0)
1      18050  270882  0.0625   (18050.0/288932.0)
Total  59814  430592  0.3625   (177760.0/490406.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.393545     0.752948  296
max f2                       0.173216     0.878568  375
max f0point5                 0.566156     0.707899  192
max accuracy                 0.518409     0.660983  222
max precision                0.891191     0.90411   2
max recall                   0.0133695    1         397
max specificity              0.902611     0.999955  0
max absolute_mcc             0.567812     0.289006  191
max min_per_class_accuracy   0.587697     0.64591   179
max mean_per_class_accuracy  0.584291     0.646141  181
max tns                      0.902611     201465    0
max fns                      0.902611     288864    0
max fps                      0.00507709   201474    399
max tps                      0.0133695    288932    397
max tnr                      0.902611     0.999955  0
max fnr                      0.902611     0.999765  0
max fpr                      0.00507709   1         399
max tpr                      0.0133695    1         397

Gains/Lift Table: Avg response rate: 58.92 %, avg score: 58.92 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100019                   0.865148           1.51391   1.51391            0.891947         0.876431  0.891947                    0.876431            0.015142        0.015142                   51.3907   51.3907            0.0125114
2        0.0200018                   0.851743           1.49137   1.50264            0.87867          0.858142  0.885309                    0.867288            0.0149135       0.0300555                  49.1373   50.2641            0.0244717
3        0.0300017                   0.841137           1.46265   1.48931            0.861746         0.846229  0.877455                    0.860269            0.0146263       0.0446818                  46.2646   48.931             0.0357327
4        0.0400015                   0.831798           1.44603   1.47849            0.851958         0.836424  0.871081                    0.854308            0.0144601       0.0591419                  44.6033   47.8491            0.0465895
5        0.0500014                   0.8237             1.43911   1.47062            0.847879         0.827692  0.866441                    0.848985            0.0143909       0.0735329                  43.9111   47.0615            0.0572777
6        0.100001                    0.789869           1.39246   1.43154            0.820392         0.806019  0.843417                    0.827503            0.0696219       0.143155                   39.2455   43.1536            0.105041
7        0.15                        0.762827           1.33556   1.39954      

##### Train Predictions

In [55]:
col_tuple_pass= ["unique_id", 'client', "wt2", 'fico_pb', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts']

In [56]:
def get_risk_rank(model, data, data_h2o, target, cols_tuple, score_col= 'p1', weight= 'RECWGT', asc= False):
    
    if data_h2o!= None:
        predict = model.predict(data_h2o)
    else:
        predict = model.predict(h2o.H2OFrame(data))
    p =predict.as_data_frame()
    dd = data.loc[:, col_tuple_pass]
    dd = pd.DataFrame(dd)
    dd = dd.reset_index()
    dd['RECWGT'] = dd["wt2"]
    dd['conwt'] = 1
    dd[target] = dd[bad_col] 
    pred = pd.concat([dd.loc[:, cols_tuple + ['RECWGT', 'conwt']],p],axis=1)
    print(pred[weight].sum(),((pred[weight]*pred[target]).sum()),pred[weight].max())

    data_ks_cap5= rnkOrderingN(pred, target, weight, score_col, 5, asc)
    display(data_ks_cap5[0])
    print(data_ks_cap5[1])
    # data_ks_cap10= rnkOrderingN(pred, target, weight, score_col, 10, asc)
    # display(data_ks_cap10[0])
    # print(data_ks_cap10[1])

In [44]:
get_risk_rank(model= xgbm_ri_auto_model, data= Train, data_h2o= train_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
490406 288932.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.739391,0.913600,0.795436,98082,18753.0,79329.0,80.88,20.0,9.31,27.46,18.15
1,0.649574,0.739389,0.695829,98081,29187.0,68894.0,70.24,40.0,23.79,51.30,27.51
2,0.551245,0.649573,0.599759,98081,38974.0,59107.0,60.26,60.0,43.14,71.76,28.62
3,0.447536,0.551245,0.502173,98081,49911.0,48170.0,49.11,80.0,67.91,88.43,20.52
4,0.001273,0.447534,0.352648,98081,64649.0,33432.0,34.09,100.0,100.00,100.00,0.00


{'Gini': 37.92, 'KS': 28.62}


In [45]:
get_risk_rank(model= xgbm_ri_auto_model, data= Test, data_h2o= test_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
210174 123941.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.739640,0.911760,0.795606,42035,7982.0,34053.0,81.01,20.0,9.26,27.48,18.22
1,0.649533,0.739639,0.695812,42035,12516.0,29519.0,70.22,40.0,23.77,51.29,27.52
2,0.551907,0.649529,0.600202,42035,16679.0,25356.0,60.32,60.0,43.11,71.75,28.64
3,0.448631,0.551905,0.503053,42035,21461.0,20574.0,48.94,80.0,68.00,88.35,20.35
4,0.002095,0.448629,0.353701,42034,27595.0,14439.0,34.35,100.0,100.00,100.00,0.00


{'Gini': 37.89, 'KS': 28.64}


#### Reading OOT Data

In [30]:
Oot= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_OOT_Data_220324.csv')
Oot.shape

(123632, 342)

In [31]:
Oot['Race']= Oot['Borrower_Race_Prim_BISG']
Oot['Gender']= Oot['borrower_gender']
Oot['age_cuts']= np.where(Oot['applicant_age'].isnull(), 'missing',
                         np.where(Oot['applicant_age']< 62, 'Age below 62',
                                  np.where(Oot['applicant_age']>= 62, 'Age 62 and over', 'missing')))

Oot['age_cuts'].value_counts(dropna= False), Oot['applicant_age'].describe()

(age_cuts
 Age below 62       109758
 Age 62 and over     13514
 missing               360
 Name: count, dtype: int64,
 count    123272.000000
 mean         42.586383
 std          14.705533
 min           0.000000
 25%          30.158459
 50%          41.880472
 75%          52.601374
 max          99.983584
 Name: applicant_age, dtype: float64)

In [32]:
Train.shape, Test.shape, Oot.shape

((490406, 145), (210174, 145), (123632, 345))

In [33]:
Oot['Race'].value_counts(dropna= False), Oot['age_cuts'].value_counts(dropna= False), Oot['Gender'].value_counts(dropna= False)

(Race
 white       68262
 missing     33191
 black       10754
 hispanic     9649
 api          1613
 aian          143
 multiple       20
 Name: count, dtype: int64,
 age_cuts
 Age below 62       109758
 Age 62 and over     13514
 missing               360
 Name: count, dtype: int64,
 Gender
 M    67879
 F    46533
 I     9220
 Name: count, dtype: int64)

In [34]:
Oot[all_cols].head(2)

,unique_id,client,app_date,approve_flag,funded_flag,fico_pb,fico_cb,bad_flag,product,auto_score,bankcard_score,Race,Gender,age_cuts,assetpropevercount,sourcenonderogcount06month,addronfilecount,addrinputcountyratio,assetpropnewestsaleprice,addrcurrenttaxvalue,addrinputavmvalue,addrinputavmvalue60month,addrinputlastsaleprice,addrinputtaxmarketvalue,sourcecredheadertimeoldest,addrinputphonecount,assetpropcurrenttaxtotal,addrcurrentavmvalue,assetproppurchasetimeoldest,addrpreviouslengthofres,addrinputlengthofres,addrcurrenttaxmarketvalue,addrprevioustimenewest,addrcurrenttractratio,sourcenonderogcount12month,assetpropcurrentcount,addrinputsubjectcount,sourcenonderogcount,addrinputtaxvalue,assetpropeversoldcount,businessassociationtimeoldest,addrinputtractratio,derogtimenewest,addrchangecount24month,addrinputtimelastsale,educationinstitutionprivate,addrinputblockratio,businesstitleleadership,addrcurrentlengthofres,addrcurrentavmratio12monthprior,addrcurrentblockratio,derogcount,profliccount,addrinputavmratio60monthprior,addrcurrentcountyratio,assetpropsaletimeoldest,subjectrecordtimeoldest,addrcurrenttimenewest,proflictypecategory,criminalnonfelonytimenewest,lienjudgmentdollartotal,addrcurrentlastsalesprice,addrinputtimenewest,evictiontimenewest,bankruptcycount,addrcurrenttimelastsale,addrinputtimeoldest,addrlastmovetaxratiodiff,criminalnonfelonycount,sourcenonderogcount03month,addrchangecount03month,addrchangecount06month,addrchangecount12month,addrchangecount60month,addrcurrentavmratio60monthprior,addrcurrentavmvalue12month,addrcurrentavmvalue60month,addrcurrentcorrectional,addrcurrentdeedmailing,addrcurrentphoneservice,addrcurrentsubjectowned,addrcurrenttimeoldest,addrinputavmratio12monthprior,addrinputavmvalue12month,addrinputdeedmailing,addrinputphoneservice,addrinputproblems,addrinputsubjectowned,addrlastmoveecontrajectory,addronfilecollege,addronfilecorrectional,addronfilehighrisk,addrpreviouscorrectional,addrprevioussubjectowned,addrprevioustimeoldest,alertregulatorycondition,assetownership,assetpersonal,assetpersonalcount,assetprop,assetpropnewestmortgagetype,assetproppurchasecount12month,assetproppurchasetimenewest,assetpropsalepurchaseratio,assetpropsaletimenewest,assetpropsoldcount12month,bankruptcychapter,bankruptcycount24month,bankruptcydismissed24month,bankruptcystatus,bankruptcytimenewest,businessassociation,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount12month,derogcount12month,educationattendance,educationevidence,educationinstitutionrating,educationprogramattended,evictioncount,evictioncount12month,inquiryauto12month,inquirybanking12month,inquirycollections12month,inquirynonshortterm12month,inquiryshortterm12month,inquirytelcom12month,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentcourtcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmentsmallclaimscount,lienjudgmenttaxcount,lienjudgmenttimenewest,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,sourcecredheadertimenewest,sourcevoterregistration,subjectnewestrecord12month,subjectrecordtimenewest
0,70325,Arkansas,2020-06-13,1.0,1.0,695.0,NaN,0.0,Auto,712.0,728.0,missing,M,Age below 62,2.0,1.0,7.0,0.36,140000.0,0.0,23338.0,18638.0,19000.0,44950.0,389.0,0.0,8990.0,-1.0,220.0,219.0,219.0,0.0,1.0,0.00,1.0,1.0,-1.0,5.0,8990.0,1.0,-1.0,0.66,68.0,0.0,-1.0,-1.0,0.49,-1.0,105.0,-1.00,0.00,2.0,0.0,1.25,0.00,-1.0,389.0,1.0,-1.0,-1.0,2844.0,-1.0,1.0,-1.0,0.0,-1.0,220.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.00,-1.0,-1.0,0.0,-1.0,0.0,-1.0,106.0,1.10,21294.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,220.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.0,220.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,68.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1,454851,Tower,2018-05-01,0.0,NaN,706.0,NaN,NaN,Auto,804.0,863.0,missing,I,Age below 62,2.0,1.0,5.0,1.11,-1.0,282633.0,318718.0,309975.0,379203.0,288700.0,424.0,1.0,285633.0,318718.0,27

In [35]:
Oot= Oot[all_cols]

for var in var_to_cat:
    Oot[var]= Oot[var].astype(int).astype(str)

In [36]:
Oot['wt2']= 1
oot_h2o = h2o.H2OFrame(Oot)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [53]:
get_risk_rank(model= xgbm_ri_auto_model, data= Oot, data_h2o= oot_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
123632 72832.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.740106,0.909830,0.795684,24727,4873.0,19854.0,80.29,20.0,9.59,27.26,17.67
1,0.650460,0.740098,0.696502,24726,7301.0,17425.0,70.47,40.0,23.96,51.18,27.22
2,0.551574,0.650459,0.600519,24727,9814.0,14913.0,60.31,60.0,43.28,71.66,28.38
3,0.447898,0.551570,0.502781,24726,12582.0,12144.0,49.11,80.0,68.05,88.33,20.28
4,0.002207,0.447898,0.353273,24726,16230.0,8496.0,34.36,100.0,100.00,100.00,0.00


{'Gini': 37.419999999999995, 'KS': 28.38}


In [54]:
var_imp = xgbm_ri_auto_model._model_json['output']['variable_importances'].as_data_frame()
var_imp['Var_unique'] = var_imp['variable'].str.split('.', expand=True)[0]
var_imp_list_baseline1 = list(var_imp['Var_unique'].unique())

print(var_imp_list_baseline1, len(var_imp_list_baseline1))

['inquiryauto12month', 'derogtimenewest', 'addrinputtaxmarketvalue', 'assetownership', 'addrinputsubjectowned', 'assetpropevercount', 'assetpropcurrenttaxtotal', 'inquiryshortterm12month', 'sourcenonderogcount', 'sourcenonderogcount12month', 'addrinputavmvalue', 'inquirynonshortterm12month', 'addrinputcountyratio', 'addrprevioustimeoldest', 'addrinputavmvalue12month', 'addrinputlengthofres', 'assetpropnewestmortgagetype', 'subjectrecordtimeoldest', 'addronfilecount', 'addrcurrentavmvalue60month', 'addrcurrentavmvalue12month', 'inquirycollections12month', 'sourcecredheadertimeoldest', 'assetpropnewestsaleprice', 'addrinputsubjectcount', 'bankruptcycount24month', 'addrinputtimenewest', 'educationevidence', 'addrinputproblems', 'addrcurrentphoneservice', 'bankruptcychapter', 'derogcount12month', 'addrcurrentcountyratio', 'addrcurrenttaxmarketvalue', 'alertregulatorycondition', 'educationprogramattended', 'assetpersonal', 'inquirytelcom12month', 'addrcurrentlengthofres', 'addrcurrentavmrat

In [55]:
var_imp

,variable,relative_importance,scaled_importance,percentage,Var_unique
0,inquiryauto12month,409511.281250,1.000000,0.135389,inquiryauto12month
1,derogtimenewest,352563.281250,0.860937,0.116561,derogtimenewest
2,addrinputtaxmarketvalue,251068.062500,0.613092,0.083006,addrinputtaxmarketvalue
3,assetownership,247591.593750,0.604603,0.081856,assetownership
4,addrinputsubjectowned,147949.312500,0.361283,0.048914,addrinputsubjectowned
5,assetpropevercount,144109.328125,0.351906,0.047644,assetpropevercount
6,assetpropcurrenttaxtotal,139378.375000,0.340353,0.046080,assetpropcurrenttaxtotal
7,inquiryshortterm12month,130606.742188,0.318933,0.043180,inquiryshortterm12month
8,sourcenonderogcount,123310.273438,0.301116,0.040768,sourcenonderogcount
9,sourcenonderogcount12month,117046.593750,0.285820,0.038697,sourcenonderogcount12month


## HyperOpt

In [56]:
# sample_1= sample.sample(frac= 0.1, random_state= 101)
# sample_1.shape

In [57]:
# sample_1_h2o= h2o.H2OFrame(sample_1)
# sample_1_h2o.shape, sample_1.shape

In [58]:
# sample_1_h2o[bad_col]= sample_1_h2o[bad_col].asfactor()

In [37]:
train_h2o[bad_col] = train_h2o[bad_col].asfactor()
test_h2o[bad_col] = test_h2o[bad_col].asfactor()
oot_h2o[bad_col] = oot_h2o[bad_col].asfactor()
# col_list = var_imp_list_baseline1
col_list= ['inquiryauto12month', 'derogtimenewest', 'addrinputtaxmarketvalue', 'assetownership', 'addrinputsubjectowned', 'assetpropevercount', 'assetpropcurrenttaxtotal', 'inquiryshortterm12month', 'sourcenonderogcount', 'sourcenonderogcount12month', 'addrinputavmvalue', 'inquirynonshortterm12month', 'addrinputcountyratio', 'addrprevioustimeoldest', 'addrinputavmvalue12month', 'addrinputlengthofres', 'assetpropnewestmortgagetype', 'subjectrecordtimeoldest', 'addronfilecount', 'addrcurrentavmvalue60month', 'addrcurrentavmvalue12month', 'inquirycollections12month', 'sourcecredheadertimeoldest', 'assetpropnewestsaleprice', 'addrinputsubjectcount', 'bankruptcycount24month', 'addrinputtimenewest', 'educationevidence', 'addrinputproblems', 'addrcurrentphoneservice', 'bankruptcychapter', 'derogcount12month', 'addrcurrentcountyratio', 'addrcurrenttaxmarketvalue', 'alertregulatorycondition', 'educationprogramattended', 'assetpersonal', 'inquirytelcom12month', 'addrcurrentlengthofres', 'addrcurrentavmratio60monthprior', 'bankruptcytimenewest', 'inquirybanking12month', 'addrinputavmvalue60month', 'addrinputtaxvalue', 'derogcount', 'bankruptcycount', 'evictiontimenewest', 'lienjudgmenttaxcount', 'lienjudgmentcourtcount', 'addrprevioustimenewest', 'assetpropsalepurchaseratio', 'proflictypecategory', 'assetpropeversoldcount', 'addrcurrenttimenewest', 'addrinputtimelastsale', 'addrcurrenttaxvalue', 'lienjudgmentothercount', 'assetpersonalcount', 'addrlastmovetaxratiodiff', 'assetpropsaletimenewest', 'addrinputlastsaleprice', 'addrcurrentavmratio12monthprior', 'addrpreviouslengthofres', 'educationinstitutionrating', 'addrcurrentavmvalue', 'bankruptcydismissed24month', 'addrinputavmratio60monthprior', 'assetproppurchasetimenewest', 'addrinputdeedmailing', 'addrcurrenttractratio', 'addrinputtractratio', 'addrcurrenttimelastsale', 'sourcevoterregistration', 'addrinputblockratio', 'addrcurrentlastsalesprice', 'assetpropsaletimeoldest', 'sourcenonderogcount06month', 'addrcurrenttimeoldest', 'addrinputavmratio12monthprior', 'lienjudgmentsmallclaimscount', 'addrinputtimeoldest', 'bankruptcystatus', 'businessassociationtimeoldest', 'assetproppurchasecount12month', 'addrprevioussubjectowned', 'lienjudgmenttimenewest']
len(col_list)
# 86
# col_list = ['inquiryauto12month', 'derogtimenewest', 'addrinputtaxmarketvalue', 'assetownership', 'addrinputsubjectowned', 'assetpropevercount', 'assetpropcurrenttaxtotal', 'inquiryshortterm12month', 'sourcenonderogcount', 'sourcenonderogcount12month', 'addrinputavmvalue', 'inquirynonshortterm12month', 'addrinputcountyratio', 'addrprevioustimeoldest', 'addrinputavmvalue12month', 'addrinputlengthofres', 'assetpropnewestmortgagetype', 'subjectrecordtimeoldest', 'addronfilecount', 'addrcurrentavmvalue60month', 'addrcurrentavmvalue12month', 'inquirycollections12month', 'sourcecredheadertimeoldest', 'assetpropnewestsaleprice', 'addrinputsubjectcount', 'bankruptcycount24month', 'addrinputtimenewest', 'educationevidence', 'addrinputproblems', 'addrcurrentphoneservice', 'bankruptcychapter', 'derogcount12month', 'addrcurrentcountyratio', 'addrcurrenttaxmarketvalue', 'alertregulatorycondition', 'educationprogramattended', 'assetpersonal', 'inquirytelcom12month', 'addrcurrentlengthofres', 'addrcurrentavmratio60monthprior', 'bankruptcytimenewest', 'inquirybanking12month', 'addrinputavmvalue60month', 'addrinputtaxvalue', 'derogcount', 'bankruptcycount', 'evictiontimenewest', 'lienjudgmenttaxcount', 'lienjudgmentcourtcount', 'addrprevioustimenewest', 'assetpropsalepurchaseratio', 'proflictypecategory', 'assetpropeversoldcount', 'addrcurrenttimenewest', 'addrinputtimelastsale', 'addrcurrenttaxvalue', 'lienjudgmentothercount', 'assetpersonalcount', 'addrlastmovetaxratiodiff', 'assetpropsaletimenewest', 'addrinputlastsaleprice', 'addrcurrentavmratio12monthprior', 'addrpreviouslengthofres', 'educationinstitutionrating', 'addrcurrentavmvalue', 'bankruptcydismissed24month', 'addrinputavmratio60monthprior', 'assetproppurchasetimenewest', 'addrinputdeedmailing', 'addrcurrenttractratio', 'addrinputtractratio', 'addrcurrenttimelastsale', 'sourcevoterregistration', 'addrinputblockratio', 'addrcurrentlastsalesprice', 'assetpropsaletimeoldest', 'sourcenonderogcount06month', 'addrcurrenttimeoldest', 'addrinputavmratio12monthprior', 'lienjudgmentsmallclaimscount', 'addrinputtimeoldest', 'bankruptcystatus', 'businessassociationtimeoldest', 'assetproppurchasecount12month', 'addrprevioussubjectowned', 'lienjudgmenttimenewest']

86

In [38]:
from IPython.display import clear_output


def get_predictions(data_h2o, bad_col, wt_col, classifier):
    predict = classifier.predict(data_h2o)
    p = predict.as_data_frame()
    dd = pd.DataFrame(data_h2o[:, unique_cols + [bad_col] + [wt_col] + ['age_cuts','Race','Gender']].as_data_frame())
    dd['RECWGT'] = dd[wt_col]
    dd['con_wt'] = 1
    dd['bad'] = dd[bad_col]
    pred = pd.concat([dd.loc[:, ('bad', "RECWGT", 'con_wt', 'age_cuts','Race','Gender')], p], axis=1)
    return pred


def find_cutoff(pred):
    sorted_pred = pred.sort_values(by='p1').reset_index(drop= True)
    cum_bad = sorted_pred['bad'].cumsum()
    bad_rate= sorted_pred['bad'].mean()
    cum_p1 = sorted_pred['p1'].cumsum()
    total_bad = cum_bad.iloc[-1]
#     total_bad= bad_rate
    cutoff_idx = (cum_p1 >= total_bad).idxmax()
    cutoff = sorted_pred.loc[cutoff_idx, 'p1']
    
    print('Pred Description')
    display(pd.DataFrame(pred[['p1']].describe()).T)
    print('Last Row')
    display(pd.DataFrame(sorted_pred.iloc[-1]).T)
    print('Cutoff Index Row')
    display(pd.DataFrame(sorted_pred.iloc[cutoff_idx]).T)
    
    print({'total_bad': total_bad, 'bad_rate': bad_rate, 'cutoff': cutoff, 'total_predicted': cum_p1.iloc[-1], 'max_prob': pred['p1'].max()})
    
    return cutoff

def Equal_Approval_Cutoff_Barsath(pred):
    pred= pred.copy()
    actual_approved_count= pred['bad'].sum()
    pred= pred.sort_values(by= ['p1'])
    pred['cumsum']= pred['con_wt'].cumsum()
    pred['approved']= np.where(pred['cumsum']<= actual_approved_count, 1, 0)
    return pred #cutoff, original_dataset

def find_cutoff_2(pred):
    pred= pred.copy()
    print('Pred Description')
    display(pd.DataFrame(pred[['p1']].describe()).T)
    prev_min = 1
    for k in range(1,100,1):
        pred["model_approval"] = [1 if i>k/100 else 0 for i in pred['p1']]
        a,b = pred.describe().loc['mean',['model_approval','bad']]
        if prev_min < abs(a-b):
            pred["model_approval"] = [1 if i>(k-1)/100 else 0 for i in pred['p1']]
            break
        else:
            prev_min = abs(a-b)
    print({'cutoff': (k-1)/100})
    return (k-1)/100

def air_calc(pred,prefix):
    air = {}
    target_control_dict = {
        'age_cuts': {
            'target': ['Age 62 and over'],
            'control': ['Age below 62']
        },
        'Race': {
            'target': ['black', 'hispanic'],
            'control': ['white']
        },
        'Race': {
            'target': ['black'],
            'control': ['white']
        },
        'Race': {
            'target': ['hispanic'],
            'control': ['white']
        },
        'Gender': {
            'target': ['F'],
            'control': ['M']
        },
        # 'mmct_tag': {
        #     'target': ['mmct'],
        #     'control': ['non-mmct']
        # }
    }
    
    for var in target_control_dict:
        target_list = target_control_dict[var]['target']
        control_list = target_control_dict[var]['control']
        p_target_list= ','.join(target_list)
        ttd_target= pred[pred[var].isin(target_list)]['con_wt'].sum()
        approved_target= pred[pred[var].isin(target_list)]['approved'].multiply(pred[pred[var].isin(target_list)]['con_wt']).sum()
        ttd_control= pred[pred[var].isin(control_list)]['con_wt'].sum()
        approved_control= pred[pred[var].isin(control_list)]['approved'].multiply(pred[pred[var].isin(control_list)]['con_wt']).sum()
        app_pct_target= round(approved_target/ttd_target, 4)
        app_pct_control= round(approved_control/ttd_control, 4)
        air[prefix+var+p_target_list]= [round(app_pct_target/app_pct_control, 4)]
        
        print({'variable': var, 'ttd_target': ttd_target, 'ttd_control': ttd_control, 
               'approved_target': approved_target, 'approved_control': approved_control})
    print(air)
        
    return air

def air_calc_new(pred,prefix):
    air = {}
    target_control_dict = [
            {'age_cuts': {
                'target': ['Age 62 and over'],
                'control': ['Age below 62']
            }},
            {'Race': {
                'target': ['black', 'hispanic'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['black'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['hispanic'],
                'control': ['white']
            }},
            {'Gender': {
                'target': ['F'],
                'control': ['M']
            }},
            # 'mmct_tag': {
            #     'target': ['mmct'],
            #     'control': ['non-mmct']
            # }
    ]

    for var in target_control_dict:
        for i in var:
            target_list = var[i]['target']
            control_list = var[i]['control']
            p_target_list= ','.join(target_list) if (
                (len(target_list)<= 1) and (('black' in target_list) or ('hispanic' in target_list))
            ) else ''
            
            ttd_target= pred[pred[i].isin(target_list)]['con_wt'].sum()
            approved_target= pred[pred[i].isin(target_list)]['approved'].multiply(pred[pred[i].isin(target_list)]['con_wt']).sum()
            ttd_control= pred[pred[i].isin(control_list)]['con_wt'].sum()
            approved_control= pred[pred[i].isin(control_list)]['approved'].multiply(pred[pred[i].isin(control_list)]['con_wt']).sum()
            app_pct_target= round(approved_target/ttd_target, 4)
            app_pct_control= round(approved_control/ttd_control, 4)
            air[prefix+i+p_target_list]= [round(app_pct_target/app_pct_control, 4)]
        
            print({'variable': i, 'target_list': p_target_list, 'ttd_target': ttd_target, 'ttd_control': ttd_control, 
                   'approved_target': approved_target, 'approved_control': approved_control})
            
    print(air)
        
    return air

In [39]:
# TTD['ApproveCount'].value_counts(dropna= False)

In [40]:
def get_prediction_2(model, data, data_h2o, target, cols_tuple, score_col= 'p1', weight= 'RECWGT', asc= False):
    if data_h2o!= None:
        predict = model.predict(data_h2o)
    else:
        predict = model.predict(h2o.H2OFrame(data))
    p =predict.as_data_frame()
    dd = data.loc[:, col_tuple_pass]
    dd = pd.DataFrame(dd)
    dd = dd.reset_index()
    dd['RECWGT'] = dd["wt2"]
    dd['conwt'] = 1
    dd[target] = dd[bad_col] 
    pred = pd.concat([dd.loc[:, cols_tuple + ['RECWGT', 'conwt']],p],axis=1)
    
    return pred

In [41]:
def objective(space):
    from sklearn.metrics import roc_auc_score
    classifier = H2OXGBoostEstimator(ntrees= int(space['n_estimators']),
                            max_depth = 2 ,
                            learn_rate = space['learning_rate'],
                            min_rows = space['min_child_weight'],
                            sample_rate = space['subsample'],
                            colsample_bylevel = space['colsample_bylevel'],
                            reg_alpha = space['reg_alpha'],
                            reg_lambda = space['reg_lambda'],
                            min_split_improvement = space['gamma'],
                            colsample_bytree = space['colsample_bytree'],
                            model_id="RI_Auto_LN_V2_Hopt_1",
                            distribution = "bernoulli",
                            stopping_metric = 'logloss',
                            seed=42)
    h2o.no_progress()
    classifier.train(y=bad_col,x =col_list,  training_frame=train_h2o, weights_column =wt_col )
    h2o.show_progress()
    pred = get_prediction_2(model= classifier, data= Train, data_h2o= train_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')
    ### Finding Cutoff
    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_train = roc_auc_score(y_true, y_prob, sample_weight=weights)
    train_ks,train_gini = rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['KS'], rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['Gini']
    # train_ks_cw,train_gini_cw = rnkOrdering2_ks_code(pred, 'bad', "con_wt","p1", 5)

    pred = get_prediction_2(model= classifier, data= Test, data_h2o= test_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')
    ### Finding Cutoff
    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_test = roc_auc_score(y_true, y_prob, sample_weight=weights)
    test_ks,test_gini = rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['KS'], rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['Gini']
    # test_ks_cw,test_gini_cw = rnkOrdering2_ks_code(pred, "bad", "con_wt","p1", 5)
    
    pred = get_prediction_2(model= classifier, data= Oot, data_h2o= oot_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')
    ### Finding Cutoff
    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_oot = roc_auc_score(y_true, y_prob, sample_weight=weights)
    oot_ks,oot_gini = rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['KS'], rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['Gini']
    # test_ks_cw,test_gini_cw = rnkOrdering2_ks_code(pred, "bad", "con_wt","p1", 5)

    # print('*'*15, ' TTD ', '*'*15)
    # pred= get_predictions(ttd_h2o, bad_col, wt_col, classifier)
    # pred = Equal_Approval_Cutoff_Barsath(pred)
    # display(pd.DataFrame(pred[['p1']].describe()).T)
    # air_ttd = air_calc_new(pred,'AIR ttd ')

    KS_diff = abs(train_ks-test_ks)
    gini_diff = abs(train_gini-test_gini)
    # ks_diff_nw = abs(train_ks_cw-test_ks_cw)
    # gini_diff_nw = abs(train_gini_cw-test_gini_cw)
    
    # ks_diff_normal= (ks_diff_nw)/15
    # ks_normal= (test_ks_cw - 20)/(40-20)
    auc_diff = abs(auc_train-auc_test)#/((auc_train+auc_test)/2)
    Stability = auc_diff
    Accuracy = (1-auc_test)
    # Fairness_Race = 1-air_ttd['AIR ttd Race'][0]
#     loss= (auc_diff*0.3) + ((1-auc_test)*0.2) + ((1-air_ttd['AIR ttd Race'][0])*0.5)
    loss= KS_diff
    
    metrics = {'loss': [loss], 'KS_diff': [KS_diff], #'KS diff non-wt': [ks_diff_nw],
                   'GINI dif': [gini_diff], #'GINI diff non-wt': [gini_diff_nw],
                   'KS train wt': [train_ks], #'KS train non-wt': [train_ks_cw],
                   'KS test wt': [test_ks], #'KS test non-wt': [test_ks_cw],
                   'KS oot wt': [oot_ks], #'KS oot non-wt': [oot_ks_cw],
#                    'AIR train': [air_tr], 'AIR test': [air_te], #'AIR oot': [air_oo],
                     # 'AIR ttd': [air_ttd],
               'AUC train':[auc_train], 'AUC test':[auc_test], #'AUC oot':[auc_oot]
    #                'Approval train target': [app_pct_target_tr], 'Approval train control': [app_pct_control_tr],
    #                'Approval test target': [app_pct_target_te], 'Approval test control': [app_pct_control_te],
    #                'Approval oot target': [app_pct_target_oo], 'Approval oot control': [app_pct_control_oo]
                  }
#     metrics.update(air_tr)
#     metrics.update(air_te)
    # metrics.update(air_oo)
    # metrics.update(air_ttd)

    # clear previous output
#     clear_output(wait=True)
#     metrics_df = pd.concat([metrics_df, pd.DataFrame(metrics)])
    # display(pd.DataFrame(metrics))
#     print(f'{loss}= ({auc_diff}*0.3) + ((1-{auc_test})*0.2) + ((1-{air_ttd["AIR ttd Race"][0]})*0.5))')
                
    return_dict =  {'loss': loss, 'status': STATUS_OK ,
           'KS_diff':KS_diff,'train_ks':train_ks,'test_ks':test_ks,'oot_ks':oot_ks, 
           'gini_diff':gini_diff,'train_gini':train_gini,'test_gini':test_gini,'oot_gini':oot_gini,
           # 'KS_cw_diff':ks_diff_nw,'train_ks_cw':train_ks_cw,'test_ks_cw':test_ks_cw,#'oot_ks_cw':oot_ks_cw,
           # 'gini_diff_nw':gini_diff_nw,'train_gini_cw':train_gini_cw,'test_gini_cw':test_gini_cw,#'oot_gini_cw':oot_gini_cw,
#            'AIR_train': air_tr, 'AIR_test': air_te, 'AIR_oot': air_oo, 'AIR ttd': air_ttd,
            'AUC train':auc_train,'AUC test':auc_test,'AUC oot':auc_oot,
             'Stability':Stability, 'Accuracy':Accuracy, 
                    # 'Fairness_Race':Fairness_Race
            }
#     return_dict.update({k: v[0] for k, v in air_tr.items()})
#     return_dict.update({k: v[0] for k, v in air_te.items()})
    # return_dict.update({k: v[0] for k, v in air_oo.items()})
    # return_dict.update({k: v[0] for k, v in air_ttd.items()})
    print(metrics)
    print()
    return return_dict

In [42]:
# space = {
#     'max_depth' : 2,#hp.choice('max_depth', range(1, 3, 1)),
#     'learning_rate' : hp.quniform('learning_rate',0.01,0.2,0.01),
#     'n_estimators' : ho_scope.int(hp.quniform('n_estimators',500,3000,100)),    
#     'min_child_weight' : hp.quniform('min_child_weight', 20, 150, 20),
#     'subsample' : hp.quniform('subsample', 0.4,1,0.1),
#     'colsample_bytree' : 1,#hp.quniform('colsample_bytree', 0.4, 1, 0.1),
#     'reg_lambda': hp.quniform('reg_lambda',1,20,0.5),
#     'reg_alpha': hp.quniform('reg_alpha',0,20,0.5),
#         'gamma':hp.quniform('gamma',0,20,0.25),
#             'colsample_bylevel':hp.quniform('colsample_bylevel',0.4, 1, 0.1)}

In [43]:
space = {
    'max_depth' :2, #  hp.choice('max_depth', range(2, 3, 1)),
    'learning_rate' : hp.quniform('learning_rate',0.01,0.2,0.001),
    'n_estimators' : ho_scope.int(hp.quniform('n_estimators',800,2500,100)),
    'min_child_weight' : hp.quniform('min_child_weight', 25, 300, 5),
    'subsample' : hp.quniform('subsample', 0.1, 0.6, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'reg_lambda':  hp.quniform('reg_lambda',5,30,0.5),
    'reg_alpha': hp.quniform('reg_alpha',1,15,0.5),
    'gamma':hp.quniform('gamma',5,35,0.5),
    'colsample_bylevel':hp.quniform('colsample_bylevel',0.5, 1, 0.05),
#     'seed':42
}

In [67]:
trials = Trials()

In [ ]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=75,
            trials=trials)

print("Best: ", best)

xgboost prediction progress: |                        
███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |                        
███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |                        
███████████████████████████████████████████████████| (done) 100%
{'loss': [0.08999999999999986], 'KS_diff': [0.08999999999999986], 'GINI dif': [0.01999999999999602], 'KS train wt': [28.7], 'KS test wt': [28.79], 'KS oot wt': [28.65], 'AUC train': [0.6998900400660177], 'AUC test': [0.7000548572359786]}
xgboost prediction progress: |                                                      
███████████████████████████████████████████████████| (done) 100%                    
xgboost prediction progress: |                                                      
███████████████████████████████████████████████████| (done) 100%                    
xgboost prediction progress: |                                

In [ ]:
tr_results = pd.DataFrame(trials.results).reset_index()
tr_results['iter'] = tr_results['index']+1
# tr_results.head()

tr_vals = pd.DataFrame(trials.vals).reset_index()
tr_vals['iter'] = tr_vals['index']+1
# tr_vals.head()

trials_full = pd.merge(tr_vals.drop('index',axis=1),tr_results.drop('index',axis=1),how='left',on='iter')
trials_full.shape

In [ ]:
trials_full

In [ ]:
# trials_full.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/RI_Auto_Model/220324_LN_Auto_RI_Hopt_2.csv', index=False)

In [ ]:
# trials_full.query('train_ks_cw > 33 and train_ks_cw > test_ks_cw and train_ks_cw - test_ks_cw < 1.5').sort_values('AIR ttd Race',ascending=False)[[
#     'iter', 'KS_cw_diff', 'train_ks_cw', 'test_ks_cw', #'oot_ks_cw',
#       'gini_diff_nw', 'train_gini_cw', 'test_gini_cw',# 'oot_gini_cw',
#          'AIR ttd age_cuts', 'AIR ttd Race', 'AIR ttd Gender', 'AIR ttd Raceblack', 'AIR ttd Racehispanic',
#       #  'AIR ttd mmct_tag',
#        'Stability', 'Accuracy', 'Fairness_Race',
#       #  'Fairness_Age', 'Fairness_Gender'#, 'Fairness_MMCT_tag'
#        ]].query('`AIR ttd Race` >= 0.9 and Stability < 0.017 and Accuracy <= 0.225').sort_values('Accuracy')

In [40]:
trials_full[(trials_full['KS_diff']<= 3.5) & (trials_full['KS_cw_diff']<= 2.5) #& 
#             (np.abs(trials_full['train_ks']-trials_full['test_ks'])<= 1.5) &
#             (np.abs(trials_full['train_ks_cw']-trials_full['test_ks_cw'])<= 1.0)
           ].sort_values(by= ['train_ks_cw'], ascending= False)#.iloc[0]

,colsample_bylevel,colsample_bytree,gamma,learning_rate,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample,iter,loss,status,KS_diff,train_ks,test_ks,gini_diff,train_gini,test_gini,KS_cw_diff,train_ks_cw,test_ks_cw,gini_diff_nw,train_gini_cw,test_gini_cw,AUC train,AUC test,Stability,Accuracy,Fairness_Race,AIR ttd age_cuts,AIR ttd Race,AIR ttd Raceblack,AIR ttd Racehispanic,AIR ttd Gender
26,0.60,0.55,22.0,0.054,100.0,1700.0,15.0,29.5,0.50,27,1.04,ok,1.74,41.71,39.97,2.39,54.02,51.63,1.04,42.38,41.34,2.07,54.06,51.99,0.789781,0.775286,0.014494,0.224714,0.2485,1.5608,0.7515,0.7602,0.7484,0.9765
85,0.60,0.90,24.0,0.037,85.0,1000.0,4.0,19.0,0.40,86,1.31,ok,3.01,41.80,38.79,2.45,53.81,51.36,1.31,42.31,41.00,2.88,54.26,51.38,0.791133,0.774399,0.016734,0.225601,0.2496,1.5636,0.7504,0.7611,0.7466,0.9752
10,0.95,0.95,18.0,0.144,225.0,1500.0,5.5,7.5,0.40,11,1.57,ok,3.11,41.93,38.82,1.91,53.56,51.65,1.57,42.24,40.67,1.38,53.83,52.45,0.788008,0.773151,0.014857,0.226849,0.2376,1.5162,0.7624,0.7978,0.7499,0.9492
78,0.70,0.90,25.5,0.113,155.0,1000.0,7.0,9.0,0.45,79,0.74,ok,1.98,41.32,39.34,2.25,53.50,51.25,0.74,42.24,41.50,3.17,54.21,51.04,0.789610,0.773999,0.015611,0.226001,0.2386,1.5512,0.7614,0.8034,0.7465,0.9719
48,0.60,0.85,18.0,0.043,145.0,900.0,8.5,22.0,0.55,49,1.17,ok,1.59,41.61,40.02,2.48,53.79,51.31,1.17,42.17,41.00,3.03,54.41,51.38,0.791388,0.773698,0.017690,0.226302,0.2453,1.5823,0.7547,0.8011,0.7381,0.9768
19,0.95,0.80,27.5,0.062,90.0,2300.0,14.0,12.5,0.45,20,0.75,ok,1.44,41.21,39.77,1.56,53.47,51.91,0.75,42.09,41.34,2.28,54.06,51.78,0.790013,0.776428,0.013585,0.223572,0.2481,1.5543,0.7519,0.7629,0.7481,0.9756
3,0.80,0.70,30.5,0.115,120.0,2000.0,7.5,14.5,0.45,4,2.26,ok,2.72,41.52,38.80,2.19,53.35,51.16,2.26,42.09,39.83,3.05,54.03,50.98,0.788985,0.772336,0.016649,0.227664,0.2383,1.5864,0.7617,0.8013,0.7476,0.9735
11,0.55,0.75,10.5,0.109,165.0,1000.0,1.0,15.5,0.30,12,1.21,ok,2.76,41.89,39.13,2.34,53.86,51.52,1.21,42.05,40.84,1.38,53.77,52.39,0.789889,0.773984,0.015905,0.226016,0.2423,1.5382,0.7577,0.7908,0.7459,0.9554
37,0.55,0.85,28.0,0.146,65.0,1800.0,14.0,28.0,0.45,38,1.35,ok,3.09,41.77,38.68,2.66,53.79,51.13,1.35,42.02,40.67,1.91,53.83,51.92,0.789545,0.774123,0.015422,0.225877,0.2421,1.5523,0.7579,0.7655,0.7551,0.9654
42,0.85,0.90,23.0,0.112,145.0,1400.0,8.0,16.5,0.40,43,0.73,ok,1.80,41.39,39.59,1.82,53.23,51.41,0.73,41.91,41.18,1.85,53.57,51.72,0.789208,0.774953,0.014255,0.225047,0.2505,1.5586,0.7495,0.8131,0.7268,0.9739


In [41]:
final_trials = trials_full.sort_values(by=['train_ks','loss'], ascending=[False,True])

In [42]:
iter_chosen = 27
best_from_hpt = final_trials.query(f'iter == {iter_chosen}').iloc[0,:].to_dict()
# for item in best_from_hpt.keys():
#     print(best_from_hpt[item])
#     try:
#         best_from_hpt[item]= np.round(best_from_hpt[item], 2)
#     except:
#         continue
print(best_from_hpt)

{'colsample_bylevel': 0.6000000000000001, 'colsample_bytree': 0.55, 'gamma': 22.0, 'learning_rate': 0.054, 'min_child_weight': 100.0, 'n_estimators': 1700.0, 'reg_alpha': 15.0, 'reg_lambda': 29.5, 'subsample': 0.5, 'iter': 27, 'loss': 1.0399999999999991, 'status': 'ok', 'KS_diff': 1.740000000000002, 'train_ks': 41.71, 'test_ks': 39.97, 'gini_diff': 2.3900000000000077, 'train_gini': 54.02, 'test_gini': 51.629999999999995, 'KS_cw_diff': 1.0399999999999991, 'train_ks_cw': 42.38, 'test_ks_cw': 41.34, 'gini_diff_nw': 2.069999999999993, 'train_gini_cw': 54.059999999999995, 'test_gini_cw': 51.99, 'AUC train': 0.7897806283700867, 'AUC test': 0.7752863935229475, 'Stability': 0.014494234847139187, 'Accuracy': 0.22471360647705252, 'Fairness_Race': 0.24850000000000005, 'AIR ttd age_cuts': 1.5608, 'AIR ttd Race': 0.7515, 'AIR ttd Raceblack': 0.7602, 'AIR ttd Racehispanic': 0.7484, 'AIR ttd Gender': 0.9765}


# best hpt

In [44]:
# train_h2o[bad_col] = train_h2o[bad_col].asfactor()
# # col_list = var_imp_list_baseline1 #[i for i in cols if i not in ["bad_flag","unique_id","wt2","client"]]
# # target = bad_col

In [45]:
len(col_list)

86

In [46]:
# best_from_hpt

In [60]:
xgbm_auto_ri_best_hopt = H2OXGBoostEstimator(ntrees= 2200, #int(best_from_hpt['n_estimators']),
                            max_depth = 2 ,
                            learn_rate = 0.047, #best_from_hpt['learning_rate'],
                            min_rows = 40, # best_from_hpt['min_child_weight'],
                            sample_rate = 0.3, # best_from_hpt['subsample'],
                            colsample_bylevel = 0.65, #best_from_hpt['colsample_bylevel'],
                            reg_alpha = 12.0, #best_from_hpt['reg_alpha'],
                            reg_lambda = 20.0, #best_from_hpt['reg_lambda'],
                            min_split_improvement = 5.0, #best_from_hpt['gamma'],
                            colsample_bytree = 0.5, #best_from_hpt['colsample_bytree'],
                            model_id="xgb_auto_RI_best_hopt",
                            distribution = "bernoulli",
                            stopping_metric = 'logloss',
                            seed=42)

In [61]:
xgbm_auto_ri_best_hopt.train(y=bad_col,x =col_list,  training_frame=train_h2o, weights_column =wt_col )

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: xgb_auto_RI_best_hopt


Model Summary: 
    number_of_trees
--  -----------------
    2200

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.21196784993579226
RMSE: 0.4603996632663758
LogLoss: 0.6118435632625141
Mean Per-Class Error: 0.4215769239470487
AUC: 0.7048178362642851
AUCPR: 0.7611888864479063
Gini: 0.40963567252857014

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3879452362008717
       0      1       Error    Rate
-----  -----  ------  -------  -------------------
0      45117  156357  0.7761   (156357.0/201474.0)
1      19384  269548  0.0671   (19384.0/288932.0)
Total  64501  425905  0.3584   (175741.0/490406.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.387945     0.754152  292
max f2                       0.166061     0.878536  375
max f0point5                 0.575074     0.711138  186
max accuracy                 0.506316     0.664142  226
max precision                0.916056     0.936275  2
max recall                   0.0625814    1         393
max specificity              0.93001      0.99997   0
max absolute_mcc             0.588996     0.296825  178
max min_per_class_accuracy   0.588996     0.649875  178
max mean_per_class_accuracy  0.588996     0.650601  178
max tns                      0.93001      201468    0
max fns                      0.93001      288870    0
max fps                      0.0158722    201474    399
max tps                      0.0625814    288932    393
max tnr                      0.93001      0.99997   0
max fnr                      0.93001      0.999785  0
max fpr                      0.0158722    1         399
max tpr                      0.0625814    1         393

Gains/Lift Table: Avg response rate: 58.92 %, avg score: 58.92 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100019                   0.882949           1.54643   1.54643            0.911111         0.896851  0.911111                    0.896851            0.0154673       0.0154673                  54.6434   54.6434            0.0133033
2        0.0200018                   0.867227           1.5111    1.52877            0.890294         0.874677  0.900703                    0.885765            0.0151108       0.0305781                  51.1101   52.8769            0.0257438
3        0.0300017                   0.855201           1.47753   1.51169            0.870514         0.860926  0.890641                    0.877486            0.0147751       0.0453532                  47.7528   51.169             0.0373671
4        0.0400015                   0.845297           1.46195   1.49926            0.861338         0.850105  0.883315                    0.870641            0.0146194       0.0599726                  46.1954   49.9257            0.0486113
5        0.0500014                   0.836156           1.46195   1.4918             0.861338         0.840655  0.87892                     0.864644            0.0146194       0.0745919                  46.1954   49.1796            0.0598556
6        0.100001                    0.800979           1.40734   1.44957            0.82916          0.817772  0.85404                     0.841209            0.070366        0.144958                   40.7338   44.9568            0.10943
7        0.15                        0.772448           1.34982   1.41632   

In [65]:
get_risk_rank(model= xgbm_auto_ri_best_hopt, data= Train, data_h2o= train_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
490406 288932.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.747291,0.942530,0.807094,98082,17903.0,80179.0,81.75,20.0,8.89,27.75,18.86
1,0.653810,0.747291,0.700944,98081,29122.0,68959.0,70.31,40.0,23.34,51.62,28.28
2,0.550514,0.653810,0.602373,98081,39042.0,59039.0,60.19,60.0,42.72,72.05,29.33
3,0.437581,0.550514,0.496711,98081,50214.0,47867.0,48.80,80.0,67.64,88.62,20.98
4,0.007050,0.437580,0.338753,98081,65193.0,32888.0,33.53,100.0,100.00,100.00,0.00


{'Gini': 38.98, 'KS': 29.33}


In [66]:
get_risk_rank(model= xgbm_auto_ri_best_hopt, data= Test, data_h2o= test_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
210174 123941.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.747734,0.940349,0.807152,42035,7748.0,34287.0,81.57,20.0,8.98,27.66,18.68
1,0.654038,0.747734,0.701213,42035,12408.0,29627.0,70.48,40.0,23.37,51.57,28.20
2,0.551294,0.654036,0.602791,42035,16752.0,25283.0,60.15,60.0,42.80,71.97,29.17
3,0.438653,0.551290,0.497725,42035,21506.0,20529.0,48.84,80.0,67.74,88.53,20.79
4,0.008660,0.438652,0.339881,42034,27819.0,14215.0,33.82,100.0,100.00,100.00,0.00


{'Gini': 38.74, 'KS': 29.17}


In [67]:
get_risk_rank(model= xgbm_auto_ri_best_hopt, data= Oot, data_h2o= oot_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
123632 72832.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.748130,0.941785,0.807155,24727,4668.0,20059.0,81.12,20.0,9.19,27.54,18.35
1,0.654556,0.748129,0.701641,24726,7308.0,17418.0,70.44,40.0,23.57,51.46,27.89
2,0.551073,0.654556,0.603110,24727,9810.0,14917.0,60.33,60.0,42.89,71.94,29.05
3,0.437305,0.551068,0.497104,24726,12717.0,12009.0,48.57,80.0,67.92,88.43,20.51
4,0.011698,0.437303,0.339035,24726,16297.0,8429.0,34.09,100.0,100.00,100.00,0.00


{'Gini': 38.32, 'KS': 29.05}


In [73]:
var_imp_bst = xgbm_auto_ri_best_hopt._model_json['output']['variable_importances'].as_data_frame()
# print(var_imp)
var_imp_bst['Var_unique'] = var_imp_bst['variable'].str.split('.', expand=True)[0]
var_imp_list_bst = list(var_imp_bst['Var_unique'].unique())

print(var_imp_list_bst, len(var_imp_list_bst))

['inquiryauto12month', 'derogtimenewest', 'assetownership', 'assetpropevercount', 'addrinputtaxmarketvalue', 'assetpropcurrenttaxtotal', 'inquiryshortterm12month', 'addrinputtaxvalue', 'sourcenonderogcount', 'inquirynonshortterm12month', 'derogcount', 'addrinputsubjectowned', 'sourcenonderogcount12month', 'addrinputavmvalue', 'addrprevioustimeoldest', 'addrinputavmvalue12month', 'sourcecredheadertimeoldest', 'addrinputlengthofres', 'addronfilecount', 'addrcurrenttaxmarketvalue', 'subjectrecordtimeoldest', 'addrcurrentavmvalue12month', 'addrinputcountyratio', 'addrinputavmvalue60month', 'assetpropnewestmortgagetype', 'addrcurrentavmvalue', 'addrinputtimeoldest', 'addrinputsubjectcount', 'inquirycollections12month', 'addrcurrentavmvalue60month', 'assetpropnewestsaleprice', 'addrinputproblems', 'alertregulatorycondition', 'addrinputtimenewest', 'addrinputtractratio', 'addrcurrenttimeoldest', 'addrcurrentlengthofres', 'addrpreviouslengthofres', 'educationevidence', 'addrinputblockratio', '

In [74]:
var_imp_bst

,variable,relative_importance,scaled_importance,percentage,Var_unique
0,inquiryauto12month,26269.943359,1.000000,0.111110,inquiryauto12month
1,derogtimenewest,17078.158203,0.650103,0.072233,derogtimenewest
2,assetownership,14622.291016,0.556617,0.061846,assetownership
3,assetpropevercount,13078.246094,0.497841,0.055315,assetpropevercount
4,addrinputtaxmarketvalue,12702.408203,0.483534,0.053726,addrinputtaxmarketvalue
5,assetpropcurrenttaxtotal,10826.840820,0.412138,0.045793,assetpropcurrenttaxtotal
6,inquiryshortterm12month,9228.603516,0.351299,0.039033,inquiryshortterm12month
7,addrinputtaxvalue,8465.710938,0.322258,0.035806,addrinputtaxvalue
8,sourcenonderogcount,7558.711426,0.287732,0.031970,sourcenonderogcount
9,inquirynonshortterm12month,7494.513184,0.285289,0.031699,inquirynonshortterm12month


In [70]:
var_imp_df = var_imp_bst.groupby('Var_unique').agg({'percentage':'sum'}).sort_values(by='percentage',ascending=False).reset_index()

In [71]:
var_imp_df

,Var_unique,percentage
0,inquiryauto12month,0.111110
1,derogtimenewest,0.072233
2,assetownership,0.061846
3,assetpropevercount,0.055315
4,addrinputtaxmarketvalue,0.053726
5,assetpropcurrenttaxtotal,0.045793
6,inquiryshortterm12month,0.039033
7,addrinputtaxvalue,0.035806
8,sourcenonderogcount,0.031970
9,inquirynonshortterm12month,0.031699


### Saving All Prediction Data

In [75]:
Train['p1']= xgbm_auto_ri_best_hopt.predict(train_h2o).as_data_frame()['p1']
Test['p1']= xgbm_auto_ri_best_hopt.predict(test_h2o).as_data_frame()['p1']
Oot['p1']= xgbm_auto_ri_best_hopt.predict(oot_h2o).as_data_frame()['p1']

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [76]:
Train['p1'].describe(), Test['p1'].describe(), Oot['p1'].describe()

(count    490406.000000
 mean          0.589176
 std           0.168691
 min           0.007050
 25%           0.469583
 50%           0.602481
 75%           0.724224
 max           0.942530
 Name: p1, dtype: float64,
 count    210174.000000
 mean          0.589754
 std           0.168290
 min           0.008660
 25%           0.471130
 50%           0.602825
 75%           0.724622
 max           0.940349
 Name: p1, dtype: float64,
 count    123632.000000
 mean          0.589611
 std           0.168607
 min           0.011698
 25%           0.470039
 50%           0.603269
 75%           0.724740
 max           0.941785
 Name: p1, dtype: float64)

In [77]:
Train['approve_flag'].mean(), Test['approve_flag'].mean(), Oot['approve_flag'].mean()

(0.589168974278455, 0.5897066240353231, 0.5891031448168759)

In [78]:
col_tuple_pass

['unique_id',
 'client',
 'wt2',
 'fico_pb',
 'approve_flag',
 'funded_flag',
 'bad_flag',
 'Race',
 'Gender',
 'age_cuts']

In [80]:
Train['weight']= 1/Train['p1']
Test['weight']= 1/Test['p1']
Oot['weight']= 1/Oot['p1']

Train['weight'].mean(), Test['weight'].mean(), Oot['weight'].mean()

(1.9456721224834104, 1.9429026054689111, 1.9421845748157416)

In [86]:
df_scored= pd.concat(
    [Train[['unique_id', 'client', 'fico_pb', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts', 'p1', 'weight']],
    Test[['unique_id', 'client', 'fico_pb', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts', 'p1', 'weight']],
    Oot[['unique_id', 'client', 'fico_pb', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts', 'p1', 'weight']]]
).reset_index(drop= True)

df_scored.shape

(824212, 11)

In [87]:
df_scored.head()

,unique_id,client,fico_pb,approve_flag,funded_flag,bad_flag,Race,Gender,age_cuts,p1,weight
0,61929,Arkansas,726.0,1.0,0.0,NaN,white,M,Age below 62,0.636944,1.569996
1,TCI196142,Numerica,802.0,1.0,1.0,0.0,white,M,Age below 62,0.790589,1.264879
2,81484,Arkansas,649.0,0.0,NaN,NaN,white,M,Age 62 and over,0.773726,1.292448
3,518897,Tower,623.0,0.0,NaN,NaN,black,I,Age below 62,0.487634,2.050718
4,93535,Arkansas,673.0,1.0,0.0,NaN,white,F,Age below 62,0.471618,2.120359


In [91]:
# df_scored.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/RI_Auto_Model/250324_RI_Auto_Model_Scored_Data.csv', index= False)

In [93]:
# xgbm_auto_ri_best_hopt.download_mojo('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/RI_Auto_Model/')

In [98]:
# model= h2o.import_mojo('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/RI_Auto_Model/xgb_auto_RI_best_hopt.zip')

In [99]:
# model.predict(train_h2o.head(4))

In [100]:
# del model